In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
df1 = pd.read_excel('raw_data_with_label.xlsx', sheet_name='links', index_col=0)
df1 = df1[df1['y'].notnull()]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 500
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         501 non-null    object 
 1   title        497 non-null    object 
 2   canal_nome   498 non-null    object 
 3   canal_link   498 non-null    object 
 4   view_counts  498 non-null    float64
 5   video_date   418 non-null    object 
 6   like         484 non-null    object 
 7   dislike      484 non-null    object 
 8   y            501 non-null    float64
dtypes: float64(2), object(7)
memory usage: 39.1+ KB


In [3]:
df1.head(1)

,link,title,canal_nome,canal_link,view_counts,video_date,like,dislike,y
0,https://www.youtube.com/watch?v=-0WQnwNFqJM,AI learns to Speedrun QWOP using Machine Learning,Wesley Liao,https://www.youtube.com/channel/UCair-JZ6XmtJk...,226613.0,25 de fev. de 2021,"2.729 marcações ""Gostei""","33 marcações ""Não gostei""",0.0


In [7]:
df2 = pd.read_excel('active_learning_done.xlsx', sheet_name='links', index_col=0)
df2 = df2[df2['y'].notnull()]
df2['novo'] = 1
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 543 to 1012
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         99 non-null     object 
 1   title        99 non-null     object 
 2   canal_nome   99 non-null     object 
 3   canal_link   99 non-null     object 
 4   view_counts  99 non-null     int64  
 5   video_date   99 non-null     object 
 6   like         99 non-null     object 
 7   dislike      99 non-null     object 
 8   p            99 non-null     float64
 9   y            99 non-null     int64  
 10  novo         99 non-null     int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 9.3+ KB


In [9]:
df2.head(1)

,link,title,canal_nome,canal_link,view_counts,video_date,like,dislike,p,y,novo
543,https://www.youtube.com/watch?v=HnKaaDcWOXw,Why I Don't Like Machine Learning,#benawad,https://www.youtube.com/hashtag/benawad,204970,25 de mar. de 2019,"5.736 marcações ""Gostei""","455 marcações ""Não gostei""",0.599,0,1


In [10]:
from sklearn.metrics import roc_auc_score, average_precision_score
average_precision_score(df2['y'], df2['p']), roc_auc_score(df2['y'], df2['p'])

(0.44441498640622623, 0.6406546489563567)

In [20]:
df = pd.concat([df1, df2.drop('p', axis=1)])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 1012
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         600 non-null    object 
 1   title        596 non-null    object 
 2   canal_nome   597 non-null    object 
 3   canal_link   597 non-null    object 
 4   view_counts  597 non-null    float64
 5   video_date   517 non-null    object 
 6   like         583 non-null    object 
 7   dislike      583 non-null    object 
 8   y            600 non-null    float64
 9   novo         99 non-null     float64
dtypes: float64(3), object(7)
memory usage: 51.6+ KB


In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [23]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']
df_limpo['novo'] = df['novo'].fillna(0)
df_limpo['views'] = df['view_counts']

## 1. Limpeza de data

In [24]:
columns = list(df.columns)
columns.remove('y')
columns.remove('novo')

df.dropna(subset=columns, inplace=True, how='any')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 516 entries, 0 to 1012
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         516 non-null    object 
 1   title        516 non-null    object 
 2   canal_nome   516 non-null    object 
 3   canal_link   516 non-null    object 
 4   view_counts  516 non-null    float64
 5   video_date   516 non-null    object 
 6   like         516 non-null    object 
 7   dislike      516 non-null    object 
 8   y            516 non-null    float64
 9   novo         99 non-null     float64
dtypes: float64(3), object(7)
memory usage: 44.3+ KB


In [26]:
clean_date = df['video_date'].str.extract(r'(\d+) de (\w+)\. de (\d+)')
clean_date[0] = clean_date[0].map(lambda x: "0"+ x[0] if len(x[0]) == 1 else x)

mapa_meses = {
    'jan': 'Jan',
    'fev': 'Feb',
    'mar': 'Mar',
    'abr': 'Apr',
    'mai': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'ago': 'Aug',
    'set': 'Sep',
    'out': 'Oct',
    'nov': 'Nov',
    'dez': 'Dec'
}

clean_date[1] = clean_date[1].map(mapa_meses)
clean_date = clean_date.apply(lambda x: ' '.join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format='%d %b %Y')

In [27]:
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 1012
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   596 non-null    object        
 1   novo    600 non-null    float64       
 2   views   597 non-null    float64       
 3   date    516 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 23.4+ KB


In [29]:
indexs = df_limpo['date'].loc[df_limpo['date'].dt.year < 2018].index.values

df_limpo.drop(index=indexs, axis='rows', inplace=True)
df.drop(index=indexs, axis='rows', inplace=True)

In [30]:
df_limpo.shape, df.shape

((553, 4), (469, 10))

## 2. Features

In [31]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [34]:
features = pd.DataFrame(index=df_limpo.index)

features['tempo_desde_pub'] = (pd.to_datetime('2021-03-18') - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features.drop(columns=['tempo_desde_pub'], inplace=True)

In [35]:
features.head()

,views,views_por_dia
0,226613.0,5150.295455
1,53263.0,57.457389
2,142715.0,373.599476
3,776910.0,809.281250
4,2290.0,6.010499


In [37]:
qtd = len(df_limpo['date'])
meio = qtd // 2 if qtd % 2 == 0 else (qtd // 2) + 1
date = str(pd.Timestamp(df_limpo.sort_values(by='date')['date'].values[meio]).date())
date

'2020-06-01'

In [39]:
mask_train = (df_limpo['date'] < date) & (df_limpo['novo'] == 0)
mask_val = (df_limpo['date'] >= date)

X_train, X_val = features[mask_train], features[mask_val]
y_train, y_val = y[mask_train], y[mask_val]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((219, 2), (199, 2), (219,), (199,))

In [40]:
y_train.value_counts(), y_val.value_counts()

(0.0    182
 1.0     37
 Name: y, dtype: int64,
 0.0    162
 1.0     37
 Name: y, dtype: int64)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [43]:
title_bow_train.shape

(219, 187)

In [44]:
from scipy.sparse import hstack

X_train_wtitle = hstack([X_train, title_bow_train])
X_val_wtitle = hstack([X_val, title_bow_val])

In [45]:
X_train_wtitle.shape, X_val_wtitle.shape

((219, 189), (199, 189))

In [46]:
mdl = RandomForestClassifier(n_estimators=100, random_state=0, class_weight='balanced')
mdl.fit(X_train_wtitle, y_train)

RandomForestClassifier(class_weight='balanced', random_state=0)

In [47]:
p = mdl.predict_proba(X_val_wtitle)[:,1]

In [48]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [49]:
average_precision_score(y_val, p), roc_auc_score(y_val, p)

(0.27014332029164356, 0.6586586586586587)

## Aumentando treino

In [50]:
mask_train = (df_limpo['date'] < date)
mask_val = (df_limpo['date'] >= date)  & (df_limpo['novo'] == 0)

X_train, X_val = features[mask_train], features[mask_val]
y_train, y_val = y[mask_train], y[mask_val]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

X_train_wtitle = hstack([X_train, title_bow_train])
X_val_wtitle = hstack([X_val, title_bow_val])

mdl = RandomForestClassifier(n_estimators=100, random_state=0, class_weight='balanced')
mdl.fit(X_train_wtitle, y_train)

p = mdl.predict_proba(X_val_wtitle)[:,1]

average_precision_score(y_val, p), roc_auc_score(y_val, p)

(0.2920965222523041, 0.6908602150537635)

In [51]:
mask_train = (df_limpo['date'] < date)
mask_val = (df_limpo['date'] >= date)

X_train, X_val = features[mask_train], features[mask_val]
y_train, y_val = y[mask_train], y[mask_val]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

X_train_wtitle = hstack([X_train, title_bow_train])
X_val_wtitle = hstack([X_val, title_bow_val])

mdl = RandomForestClassifier(n_estimators=100, random_state=0, class_weight='balanced')
mdl.fit(X_train_wtitle, y_train)

p = mdl.predict_proba(X_val_wtitle)[:,1]

average_precision_score(y_val, p), roc_auc_score(y_val, p)

(0.2820418882877961, 0.6795962629295963)